# Gmail API - Jupyter Notebook
Envío y gestión de correos usando Gmail API desde Jupyter Notebook

## 1. Instalación de dependencias

In [ ]:
!pip install google-auth-oauthlib google-auth-httplib2 google-api-python-client python-dotenv

## 2. Configuración e imports

In [ ]:
import os
import base64
import json
from email.mime.text import MIMEText
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from dotenv import load_dotenv

# Cargar variables de entorno
load_dotenv()

# Scopes de la API
SCOPES = [
    'https://www.googleapis.com/auth/gmail.send',
    'https://www.googleapis.com/auth/gmail.readonly'
]

print("✅ Módulos importados y configuración cargada")

## 3. Función de autenticación

In [ ]:
def get_gmail_service():
    """Obtiene el servicio de Gmail autenticado"""
    creds = None
    
    # Verificar variables de entorno requeridas
    required_vars = ['GOOGLE_CLIENT_ID', 'GOOGLE_CLIENT_SECRET', 'GMAIL_REFRESH_TOKEN']
    for var in required_vars:
        if not os.getenv(var):
            print(f"❌ Falta variable de entorno: {var}")
            return None
    
    # El archivo token.json almacena los tokens de acceso y actualización
    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file('token.json', SCOPES)
    
    # Si no hay credenciales válidas, usa el refresh token
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            # Usar refresh token desde variables de entorno
            refresh_token = os.getenv('GMAIL_REFRESH_TOKEN')
            client_id = os.getenv('GOOGLE_CLIENT_ID')
            client_secret = os.getenv('GOOGLE_CLIENT_SECRET')
            
            creds = Credentials(
                token=None,
                refresh_token=refresh_token,
                token_uri='https://oauth2.googleapis.com/token',
                client_id=client_id,
                client_secret=client_secret,
                scopes=SCOPES
            )
            # Refrescar el token
            creds.refresh(Request())
        
        # Guardar credenciales para próxima vez
        with open('token.json', 'w') as token:
            token.write(creds.to_json())
    
    return build('gmail', 'v1', credentials=creds)

print("✅ Función de autenticación definida")

## 4. Funciones auxiliares

In [ ]:
def create_message(to, subject, body):
    """Crea un mensaje MIME para enviar"""
    message = MIMEText(body, 'plain', 'utf-8')
    message['to'] = to
    message['subject'] = subject
    return {
        'raw': base64.urlsafe_b64encode(
            message.as_bytes()
        ).decode('ascii')
    }

def decode_message_body(data):
    """Decodifica el cuerpo del mensaje desde base64"""
    if 'data' in data:
        return base64.urlsafe_b64decode(data['data']).decode('utf-8')
    return ''

def get_message_text_parts(parts):
    """Extrae el texto de las partes del mensaje"""
    text_parts = []
    for part in parts:
        if part['mimeType'] == 'text/plain':
            text_parts.append(decode_message_body(part['body']))
        elif part['mimeType'] == 'text/html':
            text_parts.append(decode_message_body(part['body']))
        elif 'parts' in part:
            # Mensajes multipart
            text_parts.extend(get_message_text_parts(part['parts']))
    return text_parts

print("✅ Funciones auxiliares definidas")

## 5. Función para enviar correos

In [ ]:
def send_email(to=None, subject=None, body=None):
    """Envía un correo usando Gmail API"""
    try:
        service = get_gmail_service()
        if not service:
            print("❌ No se pudo obtener el servicio de Gmail")
            return None
        
        # Valores por defecto
        to = to or os.getenv('GMAIL_TO', 'jlaybar@dominio.com')
        subject = subject or os.getenv('GMAIL_SUBJECT', 'Prueba desde Jupyter Notebook')
        body = body or os.getenv('GMAIL_BODY', 'Hola! 👋 Este correo fue enviado desde Jupyter Notebook usando la API de Gmail.')
        
        # Crear y enviar mensaje
        message = create_message(to, subject, body)
        result = service.users().messages().send(
            userId='me', 
            body=message
        ).execute()
        
        print(f'✅ Correo enviado exitosamente!')
        print(f'   ID: {result["id"]}')
        print(f'   Para: {to}')
        print(f'   Asunto: {subject}')
        
        return result
        
    except HttpError as err:
        error_detail = err.content.decode('utf-8') if err.content else str(err)
        print(f'❌ Error enviando correo: {error_detail}')
        return None
    except Exception as err:
        print(f'❌ Error inesperado: {str(err)}')
        return None

print("✅ Función de envío definida")

## 6. Función para listar mensajes

In [ ]:
def list_messages(max_results=10, label_ids='INBOX'):
    """Lista los últimos mensajes del buzón"""
    try:
        service = get_gmail_service()
        if not service:
            print("❌ No se pudo obtener el servicio de Gmail")
            return None
        
        # Obtener lista de mensajes
        result = service.users().messages().list(
            userId='me',
            maxResults=max_results,
            labelIds=label_ids
        ).execute()
        
        messages = result.get('messages', [])
        messages_list = []
        
        print(f"📧 Obteniendo detalles de {len(messages)} mensajes...")
        
        # Obtener detalles de cada mensaje
        for i, msg in enumerate(messages):
            print(f"   Procesando mensaje {i+1}/{len(messages)}...")
            
            message_detail = service.users().messages().get(
                userId='me', 
                id=msg['id'],
                format='metadata',
                metadataHeaders=['Subject', 'From', 'Date']
            ).execute()
            
            # Extraer headers importantes
            headers = {h['name']: h['value'] for h in message_detail.get('payload', {}).get('headers', [])}
            
            messages_list.append({
                'id': msg['id'],
                'threadId': msg.get('threadId'),
                'snippet': message_detail.get('snippet', ''),
                'subject': headers.get('Subject', ''),
                'from': headers.get('From', ''),
                'date': headers.get('Date', ''),
                'labelIds': message_detail.get('labelIds', [])
            })
        
        print(f"✅ Se obtuvieron {len(messages_list)} mensajes")
        return messages_list
        
    except HttpError as err:
        print(f'❌ Error de Gmail API: {err}')
        return None
    except Exception as err:
        print(f'❌ Error inesperado: {str(err)}')
        return None

print("✅ Función para listar mensajes definida")

## 7. Función para obtener mensaje específico

In [ ]:
def get_message_details(message_id):
    """Obtiene el cuerpo completo de un mensaje específico"""
    try:
        service = get_gmail_service()
        if not service:
            print("❌ No se pudo obtener el servicio de Gmail")
            return None
        
        # Obtener mensaje completo
        message = service.users().messages().get(
            userId='me', 
            id=message_id,
            format='full'
        ).execute()
        
        # Extraer headers
        headers = {h['name']: h['value'] for h in message.get('payload', {}).get('headers', [])}
        
        # Extraer cuerpo del mensaje
        body_text = ''
        payload = message.get('payload', {})
        
        if 'parts' in payload:
            # Mensaje multipart
            text_parts = get_message_text_parts(payload['parts'])
            body_text = '
'.join(text_parts)
        elif 'body' in payload and 'data' in payload['body']:
            # Mensaje simple
            body_text = decode_message_body(payload['body'])
        
        message_details = {
            'id': message['id'],
            'threadId': message.get('threadId'),
            'snippet': message.get('snippet', ''),
            'subject': headers.get('Subject', ''),
            'from': headers.get('From', ''),
            'to': headers.get('To', ''),
            'date': headers.get('Date', ''),
            'body': body_text,
            'labelIds': message.get('labelIds', [])
        }
        
        print(f"✅ Mensaje obtenido exitosamente")
        return message_details
        
    except HttpError as err:
        print(f'❌ Error de Gmail API: {err}')
        return None
    except Exception as err:
        print(f'❌ Error inesperado: {str(err)}')
        return None

print("✅ Función para obtener mensaje definida")

## 8. Ejemplos de uso

### 8.1 Enviar un correo de prueba

In [ ]:
# Enviar correo de prueba
result = send_email(
    to="destinatario@ejemplo.com",  # Cambia por un email real
    subject="Prueba desde Jupyter Notebook",
    body="¡Hola! Este es un correo de prueba enviado desde Jupyter Notebook usando Gmail API."
)

if result:
    print("Correo enviado con éxito!")
else:
    print("Error al enviar el correo")

### 8.2 Listar últimos mensajes

In [ ]:
# Listar últimos 5 mensajes
messages = list_messages(max_results=5)

if messages:
    print("\n📬 Últimos mensajes:")
    print("-" * 80)
    for i, msg in enumerate(messages):
        print(f"{i+1}. De: {msg['from']}")
        print(f"   Asunto: {msg['subject']}")
        print(f"   Fecha: {msg['date']}")
        print(f"   ID: {msg['id']}")
        print(f"   Snippet: {msg['snippet'][:100]}...")
        print("-" * 80)

### 8.3 Obtener mensaje específico

In [ ]:
# Obtener detalles de un mensaje específico (usa un ID de la lista anterior)
if messages:
    message_id = messages[0]['id']  # Usar el primer mensaje de la lista
    message_details = get_message_details(message_id)
    
    if message_details:
        print(f"\n📧 Mensaje: {message_details['subject']}")
        print("=" * 80)
        print(f"De: {message_details['from']}")
        print(f"Para: {message_details['to']}")
        print(f"Fecha: {message_details['date']}")
        print(f"ID: {message_details['id']}")
        print("-" * 80)
        print("CUERPO:")
        print("-" * 80)
        print(message_details['body'][:500] + "..." if len(message_details['body']) > 500 else message_details['body'])
else:
    print("Primero ejecuta la celda de listar mensajes para obtener un ID válido")

## 9. Función para mostrar resumen del buzón

In [ ]:
def mailbox_summary():
    """Muestra un resumen del buzón de Gmail"""
    try:
        service = get_gmail_service()
        if not service:
            return None
        
        # Obtener perfiles
        profile = service.users().getProfile(userId='me').execute()
        
        # Obtener conteo de mensajes por etiqueta
        labels = service.users().labels().list(userId='me').execute()
        
        print("📊 RESUMEN DEL BUZÓN")
        print("=" * 50)
        print(f"Email: {profile['emailAddress']}")
        print(f"Mensajes totales: {profile.get('messagesTotal', 'N/A')}")
        print(f"Hilos totales: {profile.get('threadsTotal', 'N/A')}")
        print("\nEtiquetas principales:")
        
        for label in labels.get('labels', [])[:10]:  # Mostrar solo las primeras 10
            print(f"  • {label['name']}: {label.get('messagesTotal', 0)} mensajes")
        
        return profile
        
    except Exception as err:
        print(f"❌ Error obteniendo resumen: {str(err)}")
        return None

# Ejecutar resumen
mailbox_summary()

## 10. Configuración de variables de entorno

Crea un archivo `.env` en la misma carpeta con:

```env
GOOGLE_CLIENT_ID=tu_client_id_aqui
GOOGLE_CLIENT_SECRET=tu_client_secret_aqui
GMAIL_REFRESH_TOKEN=tu_refresh_token_aqui
GMAIL_TO=destinatario@ejemplo.com
GMAIL_SUBJECT=Asunto por defecto
GMAIL_BODY=Cuerpo por defecto
```

O configura las variables directamente:

In [ ]:
# Configuración directa (alternativa a .env)
os.environ['GOOGLE_CLIENT_ID'] = 'tu_client_id_aqui'
os.environ['GOOGLE_CLIENT_SECRET'] = 'tu_client_secret_aqui'
os.environ['GMAIL_REFRESH_TOKEN'] = 'tu_refresh_token_aqui'

print("✅ Variables de entorno configuradas")

## Resumen de funciones disponibles

- `send_email(to, subject, body)` - Enviar correo
- `list_messages(max_results, label_ids)` - Listar mensajes
- `get_message_details(message_id)` - Obtener mensaje específico
- `mailbox_summary()` - Resumen del buzón